# Packages


In [2]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json
import duden
import time
from collections import Counter
import pandas as pd
import csv

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

In [ ]:
playlist_id = '2siI2ILJEPr88yibWLgQQl'  #Playlist: "Deutsche Songs die jeder kennt ;)"
playlist_id = '37i9dQZF1DWTWEW1zqSeEj' #Playlist: Zurück in die 90er
playlist_id ='56r5qRUv3jSxADdmBkhcz7' # Playlist: Top Hits of 2022

Ausführen von getlyrics.py

## Analysis

In [30]:
with open("./all_songs.json") as f:
    songs_dict = json.load(f)

1. Spacy

Ausführen von cyspa.py

In [65]:
len(songs_dict)

3

In [10]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl

nlp = spacy.load("de_core_news_sm")
#nlp1 = spacy.load ("de_dep_news_trf")
nlp2 = spacy.load ("de_core_news_lg")

list_of_nouns, list_of_pronouns = [],[]

def spacify_with_coref(lyrics_from_song):
    doc3 = nlp2(lyrics_from_song)
    doc = nlp(lyrics_from_song)

    doc._.coref_chains = doc3._.coref_chains

    return doc

for playlist in songs_dict.keys():
    list_of_nouns_playlist, list_of_pronouns_playlist = [], []
    for song_id in tqdm(songs_dict[playlist].keys()):

        lyrics_from_song = songs_dict[playlist][song_id]["lyrics"]
        doc = nlp(lyrics_from_song)
        #doc2 = nlp1 (lyrics_from_song)
        doc3 = nlp2 (lyrics_from_song)

        # Ausgabe lediglich von Wörtern, die Nomen und Pronomen sind:
        for token in doc3: 
            if token.pos_ == "NOUN":
                list_of_nouns_playlist.append(token.text)
            elif token.pos_ == "PRON":
                list_of_pronouns_playlist.append(token.text)
            else:
                pass
    list_of_nouns.append(list_of_nouns_playlist)
    list_of_pronouns.append(list_of_pronouns_playlist)

100%|██████████| 88/88 [00:25<00:00,  3.45it/s]


List in Json

Json in Dict

In [ ]:
lon_dict = {name: list_ for name, list_ in zip(songs_dict.keys(), list_of_nouns)}
lop_dict = {name: list_ for name, list_ in zip(songs_dict.keys(), list_of_pronouns)}

In [130]:
print(f"{len(list_of_nouns)} nouns were found.")
print(f"{len(list_of_pronouns)} pronouns were found.")

3 nouns were found.
3 pronouns were found.


2. Daten sortieren



NOMEN

Json-Datei wird erstellt, um eine Liste mit allen Nomen und zugehörigen Artikeln zu bekommen. (Durchlaufzeit 28 min)

In [24]:
scrape_data = False # decide, whether to scrape full data again or read an exisiting json file.

In [27]:
if scrape_data:
    #Dictionary
    result_nomen = {}

    unknown_nomen = 0
    id = 0
    for playlist in lon_dict.keys():
        nomen_playlist = {}
        for word in tqdm(lon_dict[playlist]):
        
            try_again = True
            while try_again:
                try:
                    n = duden.get(word)
                    try_again = False
                except:
                    print(f"Error at playlist {playlist} with word {word}.")
                    time.sleep(2)
        
            if n is not None:
                nomen_playlist[str(id)] = {"WORT": n.name, "ARTIKEL": n.article}
                id += 1
            else:
                unknown_nomen += 1

        result_nomen[playlist] = nomen_playlist

        
    with open("Nomen.json", "w") as fp:
        json.dump(result_nomen, fp, indent=4)
        
else:
    with open("./Nomen.json", "r") as f:
        json.load(f)

In [25]:
with open("./Nomen.json", "r") as f:
    json.load(f)

Einzelne Json-Dateien für Nomen (m,w,n) erstellen.

In [112]:
result_nomen["Prototyp"]["0"]["WORT"]=="Kneipe"

True

In [ ]:
nomen_sortiert={}
id=0
for playlist in result_nomen.keys():
    nomen_g={}
    for word in tqdm(result_nomen[playlist]):
        if result_nomen[playlist][word]["ARTIKEL"] == "der":
            nomen_g[str(id)]= {"Maskuline Nomen": result_nomen[playlist][word]["WORT"]}
            id +=1
        elif result_nomen[playlist][word]["ARTIKEL"] == "die":
            nomen_g[str(id)]={"Feminine Nomen": result_nomen[playlist][word]["WORT"]}
            id +=1
        elif result_nomen[playlist][word]["ARTIKEL"] == "das":
            nomen_g[str(id)]={"Neutrale Nomen": result_nomen[playlist][word]["WORT"]}
            id +=1
        else:
            pass
            

    nomen_sortiert[playlist]= nomen_g
with open("Nomen_sortiert.json", "w") as fp:
    json.dump(nomen_sortiert, fp, indent=4)

# Title tbd

- Sonderzeichen in strings?

### Read files

In [3]:
gender_neu = pd.read_csv("./gender_neu_Nomen.csv",sep=";",header=None,encoding="utf8")
gender_neu = gender_neu.iloc[:, 0]
gender_neu.columns = ["Wort"]
gender_neu = list(gender_neu)

with open("./Nomen_sortiert.json", "r") as f:
    nomen_sortiert = json.load(f)

In [4]:
True if (nomen_sortiert['Prototyp']['1']['Feminine Nomen']) in gender_neu else False

False

In [10]:
complete_df = pd.DataFrame(columns=["Geschlecht","Wort","neutral"])
tmp = []
playlist='Prototyp'
for id in nomen_sortiert[playlist].keys():
    for g in nomen_sortiert[playlist][id].keys(): # geschlecht
        w = nomen_sortiert[playlist][id][g] # wort
        is_gender_neutral = True if w in gender_neu else False

        complete_df.loc[len(complete_df)] = [g, w, is_gender_neutral]
        tmp.append(g)

In [11]:
freq = Counter(tmp)
freq

Counter({'Maskuline Nomen': 1195,
         'Feminine Nomen': 1092,
         'Neutrale Nomen': 711})

In [14]:
complete_df.head()

,Geschlecht,Wort,neutral
0,Feminine Nomen,Kneipe,False
1,Feminine Nomen,Ecke,False
2,Maskuline Nomen,Abend,False
3,Feminine Nomen,Liebe,False
4,Feminine Nomen,Freundschaft,False


In [15]:
complete_df.to_json("./test.json")

In [16]:
complete_df.loc[complete_df["neutral"] == True, :]
complete_df

,Geschlecht,Wort,neutral
0,Feminine Nomen,Kneipe,False
1,Feminine Nomen,Ecke,False
2,Maskuline Nomen,Abend,False
3,Feminine Nomen,Liebe,False
4,Feminine Nomen,Freundschaft,False
5,Maskuline Nomen,Refrain,False
6,Feminine Nomen,Stadt,False
7,Neutrale Nomen,Leben,False
8,Feminine Nomen,Wand,False
9,Neutrale Nomen,Dorf,False


In [94]:
with open('Example.csv', 'w', newline = '') as csvfile:
    my_writer = csv.writer(csvfile, delimiter = ' ')
    my_writer.writerow(complete_df)

# Pronomen

In [ ]:
result_pronomen ={}

id=0
for playlist in lop_dict.keys():
    pronomen_playlist = {}
    for word in tqdm(lop_dict[playlist]):
        #männlich
        if word.lower() in ["er", "ihn", "derjenige", "derselbe","wer", "wen", "wem"]:
            pronomen_playlist[str(id)]={"Maskuline Pronomen": word}
            id +=1
        
        #weiblich
        if word.lower() in ["sie", "ihre", "dieselbe"]:
            pronomen_playlist[str(id)]={"Feminine Pronomen": word}
            id +=1
      
    result_pronomen[playlist]=pronomen_playlist
with open('Pronomen_sortiert.json', 'w') as fp:
    json.dump(result_pronomen, fp, indent=2)


In [ ]:
list_ = []
for id in result_pronomen["Songs22"].keys():
    list_.append(str(result_pronomen["Songs22"][id].keys()))
list_

In [ ]:
freq = Counter(list_)
freq

In [191]:
df = pd.read_json('Pronomen_sortiert.json')
df.to_csv('PRONOMEN.csv')

In [15]:
d=pd.read_csv('PRONOMEN.csv')
d=d.pop('Prototyp')
d.to_csv('PRONOMEN_neu.csv')

In [16]:

PD= pd.json_normalize(df['Prototyp'])
PD1= pd.json_normalize(df['Songs90er'])
PD2= pd.json_normalize(df['Songs22'])


In [ ]:
print()

In [ ]:
df1 = pd.read_json('Liste_Pronomen.json')
df1.to_csv('PRONOMEN_L.csv')

df2 = pd.read_json('Nomen_sortiert.json')
df2.to_csv('Nomen_L.csv')